In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import gefry3 as G

## Loading inputs and basic functionality

Input decks are in YAML, a simple human-readable markup language.
You can use `read_input_problem` to load a problem deck. This returns
an object representing the problem, which can be called to evalute the
detector response (more on this in a minute). 

`read_input_problem` takes two paramaters. The first is the path to the
input file and is mandatory. The second is `problem_type`, which selects
between different models. Currently `problem_type` can be one of two
options:

1. `"Simple_Problem"` - the basic type of problem, cross sections are fixed. 
2. `"Perturbable_XS_Problem"` - cross section's are variable and are provided
    as an argument at every evaluation.
        
If you do not specify a `problem_type` it will either read it from the input
file or if it is not given there it will default to `Simple_Problem`. If
`problem_type` is given to `read_input_problem` and it is present in the input
file, the value passed to `problem_type` will take priority (this will generate
a warning).

In [3]:
P = G.read_input_problem("g3_deck.yml", problem_type="Simple_Problem")

In [4]:
S0 = np.array([158., 98.]) # This is the *actual* source location, in meters

I0 = 3.214e9               # The actual source intensity, corresponds to
                           # 1 mg of Cesium-137, this value is in Becquerels
    
# These are also given in g3_deck.yml, though they aren't used by the code
# and are there just to keep things organized.

`P` is now `Simple_Problem`. You can call `P` to get the vector of detector responses.
It takes two arguments, `r`, which is a list-like (i.e., a NumPy array) of the `(x,y)`
coordinates of the source, and `I`, which is the activity of the source in Becquerels
(disintegrations / second). 

Here I generate the nominal response to the true source by calling P at the true source
location and intensity.

In [5]:
nominal = P(S0, I0)
print(nominal)

[  29.87849686   89.7422458     0.41692635  232.13770905  138.93759013
   26.94075027   48.34436879  292.29445259  196.03584566  170.2281518 ]


It's important to note: the output is a vector of bulk counts seen by each detector. This is a 
**purely deterministic** calculation using the ray tracing model we've developed. This 
does not include any background and does not include any statistical variation, we have
to add these ourselves.

Also of note - the order of the entries correspond to the detectors, *according to the
order they were given in the input file*.

Now, let's generate some simulated data that does include background. Let's start with background,
a good value for the detectors we are simulating in the Southeastern U.S. is that you
will measure a mean of 300 counts per second (CPS). However the model works with bulk counts (i.e.,
`(counts per second) * (measurement time)`), so we need to read the dwell times for the detectors 
("dwell time" is how long the detector recorded counts for). 

The dwell times are all in seconds. Again, the order here of the dwell times matches the order in the
input (they're all the same for this example, but they will usually be at least slightly different in
a real measurement). Also of note, this is assuming the background rate is constant everywhere in
the scene. This is not exactly true in reality.

In [6]:
B0 = 300 # Nominal background rate, cps

dwell = array([detector.dwell for detector in P.detectors])
B_nominal = B0 * dwell

print(dwell)
print(B_nominal)

[ 5.  5.  5.  5.  5.  5.  5.  5.  5.  5.]
[ 1500.  1500.  1500.  1500.  1500.  1500.  1500.  1500.  1500.  1500.]


We can now simulate a response by drawing a sample from `Po[nominal + B_nominal]`.

The result is a vector of *counts* recorded by each detector, which we can use as
our calibration data.

In [7]:
data = np.random.poisson(nominal + B_nominal)

print(data)

[1577 1607 1484 1777 1626 1553 1581 1785 1728 1610]
